<a href="https://colab.research.google.com/github/Paco-clod/Amazon-Recommendation-System-using-Python/blob/main/AmazonRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/amazon.csv')
data.head()

,AKM1MP6P0OYPR,0132793040,5.0,1365811200
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


In [ ]:
data.shape

(7824481, 4)

The dataset that I am using here does not have columns names, so let’s give the most appropriate names to these columns:

In [ ]:
data.columns = ['user_id', 'product_id', 'ratings', 'timestamp']
data.head()

,user_id,product_id,ratings,timestamp
0,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
1,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
2,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
3,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200
4,A1QGNMC6O1VW39,0511189877,5.0,1397433600


This dataset is very large, so i will select a sample

In [ ]:
df = data[:int(len(data) * .1)]
df.shape

(782448, 4)

In [ ]:
df.describe()

,ratings,timestamp
count,782448.000000,7.824480e+05
mean,3.981954,1.242607e+09
std,1.396133,1.163992e+08
min,1.000000,9.127296e+08
25%,3.000000,1.154650e+09
50%,5.000000,1.247270e+09
75%,5.000000,1.356653e+09
max,5.000000,1.406074e+09


In [ ]:
df.dtypes

,0
user_id,object
product_id,object
ratings,float64
timestamp,int64


In [ ]:
df.isnull().sum()

,0
user_id,0
product_id,0
ratings,0
timestamp,0


In [ ]:
df.duplicated().sum()

0

Now let’s prepare the dataset for creating a recommendation system:

In [ ]:
counts = df['user_id'].value_counts()
data = df[df['user_id'].isin(counts[counts >= 50].index)]
data.groupby('product_id')['ratings'].mean().sort_values(ascending=False)
final_ratings = data.pivot(index = 'user_id', columns ='product_id', values = 'ratings').fillna(0)

num_of_ratings = np.count_nonzero(final_ratings)
possible_ratings = final_ratings.shape[0] * final_ratings.shape[1]
density = (num_of_ratings/possible_ratings)
density *= 100
final_ratings_T = final_ratings.transpose()

grouped = data.groupby('product_id').agg({'user_id': 'count'}).reset_index()
grouped.rename(columns = {'user_id': 'score'},inplace=True)
training_data = grouped.sort_values(['score', 'product_id'], ascending = [0,1])
training_data['Rank'] = training_data['score'].rank(ascending=0, method='first')
recommendations = training_data.head()

In [ ]:
recommendations

,product_id,score,Rank
113,B00004SB92,6,1.0
1099,B00008OE6I,5,2.0
368,B00005AW1H,4,3.0
612,B0000645C9,4,4.0
976,B00007KDVI,4,5.0


Now I will write a Python function to generate recommendations based on the score of the product reviews:

In [ ]:
def recommend(id):
    recommend_products = recommendations
    recommend_products['user_id'] = id
    column = recommend_products.columns.tolist()
    column = column[-1:] + column[:-1]
    recommend_products = recommend_products[column]
    return recommend_products

recommend(5)

<ipython-input-27-a36d26906a70>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommend_products['user_id'] = id


,user_id,product_id,score,Rank
113,5,B00004SB92,6,1.0
1099,5,B00008OE6I,5,2.0
368,5,B00005AW1H,4,3.0
612,5,B0000645C9,4,4.0
976,5,B00007KDVI,4,5.0


Summary
This is how we can create an Amazon Recommender System using Python. This dataset does not have names of products in it, it only had product id so the score of the product reviews becomes the most important feature for such kinds of datasets.